# OL* vs L*

We will compare the L* algorithm by Angluin (1987) with our new OL* algorithm, which decomposes a Mealy machine into components, where each components models the behaviour of the SUL for a single output.

We ran both OL* and L* on the benchmarks from Labbaf et al. (2023). This benchmark consists of the parallel interleaving of multiple smaller automata. Note that the components of these machines can be split by their input alphabet, but not necessarily by their output alphabet. Some outputs are unique for a component, and can thus by learned efficiently by OL*, while others appear in multiple components (like the output **1**), so these will be tricky for OL* to learn.

In [151]:
import matplotlib.pyplot as plt
import pandas as pd

data_olstar = pd.read_csv("D:\\Data\\results_labbaf_olstar.csv")
data_lstar = pd.read_csv("D:\\Data\\results_labbaf_lstar.csv")

data_olstar.sum()

Model                      19418
Stages                       212
States                    148781
Short rows                111403
Inconsistent count          5068
Zero outputs count             0
Two outputs count              0
Learning queries       164259359
Learning symbols      2157248568
Testing queries         68686443
Testing symbols        746068858
dtype: int64

> Note: After making the table consistent, there were no instances were the resulting machine returns zero or multiple outputs at the same time!

In [166]:
len(data_olstar[data_olstar["Testing queries"] > 0])

8

In [165]:
len(data_lstar[data_lstar["Testing queries"] > 0])

37

> Note: All testing queries for OL* come from only 8 models. In contrast, L* needs testing queries for 37 models.

We will first analyse the performance of OL* against the performance of L*.

In [122]:
print("OL* Learning:", f"{data_olstar["Learning queries"].mean():_.2f}")
print("L* Learning: ", f"{data_lstar["Learning queries"].mean():_.2f}")

OL* Learning: 838_057.95
L* Learning:  421_086.48


In [124]:
print("OL* Testing:", f"{data_lstar["Testing queries"].mean():_.2f}")
print("L* Testing:   ", f"{data_olstar["Testing queries"].mean():_.2f}")

OL* Testing: 1_675_059.39
L* Testing:    350_441.04


In [126]:
def total_queries(df):
    return (df["Learning queries"] + df["Testing queries"]).mean()

print("OL* Total:", f"{total_queries(data_olstar).mean():_.2f}")
print("L* Total: ", f"{total_queries(data_lstar).mean():_.2f}")

OL* Total: 1_188_498.99
L* Total:  2_096_145.87


When looking at just the number of learning queries, L* outperforms OL*. However, OL* more than makes up for this with the number of testing queries needed.

## Testing queries

Many models do not require any equivalence queries to learn correctly (ignoring the last equivalence query which returns "true"). From initial testing, OL* seemed to have a big advantage for models that _do_ require equivalence queries. It requires fewer equivalence queries, which results in fewer testing queries. We will now analyse how big the advantage of OL* is for these models.

In [160]:
tq_olstar = data_olstar[data_lstar["Testing queries"] > 0]
tq_lstar = data_lstar[data_lstar["Testing queries"] > 0]

print(f"{total_queries(tq_olstar).mean():_.2f}")
print(f"{total_queries(tq_lstar).mean():_.2f}")

37
4_491_453.76
9_602_850.46


## Short prefix rows

The main benefit of using OL* is that it can decompose a full machine into smaller components, where each component models the behaviour for one output symbol. However, some models cannot be decomposed: the smallest component has as many states as the full machine. The number of states of components is reflected in the number of short prefix rows in the table during learning.

In this section, we will analyse the two possible scenarios: the number of short prefix rows needed by OL* is smaller than the number of states of the SUL, and the number of short prefix rows needed by OL* is equal to the number of states of the SUL. Note that it is impossible for OL* to require more short prefix rows that there are states in the SUL, since we only add a short prefix row when there is an output for which this row differs from all previous rows, and the number of unique rows is precisely the number states of the SUL.

#### Short prefix rows < Number of states

We will start our analysis by looking at the scenario in which OL* needs fewer short prefix rows than the number of states in the SUL. Our hypothesis would be that OL* would then need fewer queries than L*, since the model has been effectively decomposed into smaller components.

In [135]:
short_olstar = data_olstar[data_olstar["Short rows"] < data_olstar["States"]]
short_lstar = data_lstar[data_olstar["Short rows"] < data_olstar["States"]]

print(f"{total_queries(short_olstar).mean():_.2f}")
print(f"{total_queries(short_lstar).mean():_.2f}")

1_189_797.15
2_480_564.07


#### Short prefix rows = Number of states

When the number of short prefix rows needed by OL* is equal to the number of states of the SUL, we would expect OL* to either do similar to L*, since the algorithm reduces to doing L*, or to do worse, since it would require some extra queries to realise that the smallest component is the full SUL.

In [149]:
equal_olstar = data_olstar[data_olstar["Short rows"] == data_olstar["States"]]
equal_lstar = data_lstar[data_olstar["Short rows"] == data_olstar["States"]]

print(f"{total_queries(equal_olstar).mean():_.2f}")
print(f"{total_queries(equal_lstar).mean():_.2f}")

equal_olstar.mean()

1_187_001.11
1_652_586.41


Model                 8.092308e+01
Stages                1.175824e+00
States                5.552198e+02
Short rows            5.552198e+02
Inconsistent count    4.703297e+00
Zero outputs count    0.000000e+00
Two outputs count     0.000000e+00
Learning queries      4.322050e+05
Learning symbols      4.492356e+06
Testing queries       7.547961e+05
Testing symbols       8.198559e+06
dtype: float64

In [148]:
equal_lstar.mean()

Model               8.092308e+01
Stages              1.307692e+00
States              5.552198e+02
Learning queries    3.301116e+05
Learning symbols    2.969751e+06
Testing queries     1.322475e+06
Testing symbols     1.477462e+07
dtype: float64